<a href="https://colab.research.google.com/github/mahdiheidari721/AI-Internship/blob/master/House_data_MLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# import libraries
import torch
import numpy as np

In [5]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

--2022-08-15 14:16:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49082 (48K) [application/x-httpd-php]
Saving to: ‘housing.data’

housing.data        100%[===================>]  47.93K  --.-KB/s    in 0.1s    

2022-08-15 14:16:41 (331 KB/s) - ‘housing.data’ saved [49082/49082]



In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
dataset = pd.read_csv('housing.data', sep="\t")
y = dataset.iloc[:, -1].values
A=[]
for i in range(505) :
  A.append(y[i].split())
A=np.array(A)  
X = A[:, :-1]
y = A[:, -1]
############# new lines #############
y = y[:, np.newaxis]
X = X.astype('float32')
y = y.astype('float32')
####################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
batch_size=16

In [7]:
(X_train.shape, y_train.shape)

((404, 13), (404, 1))

In [8]:
############ data normaliztion #############
max_vec_x_train = np.max(X_train, axis=0)
min_vec_x_train = np.min(X_train, axis=0)

max_vec_y_train = np.max(y_train, axis=0)
min_vec_y_train = np.min(y_train, axis=0)


X_train = (X_train - min_vec_x_train)/(max_vec_x_train - min_vec_x_train)
X_test = (X_test - min_vec_x_train)/(max_vec_x_train - min_vec_x_train)

y_train = (y_train - min_vec_y_train)/(max_vec_y_train - min_vec_y_train)
y_test = (y_test - min_vec_y_train)/(max_vec_y_train - min_vec_y_train)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 32
        hidden_2 = 32
        hidden_3 = 1
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(13, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        #x = x.view(-1, 13)
        # add hidden layer, with relu activation function
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        # add dropout layer
        #x = self.dropout(x)
        x = F.sigmoid(self.fc3(x))
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=13, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [10]:
# specify loss function (categorical cross-entropy)
criterion = nn.MSELoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [14]:
xx=torch.from_numpy(X_train)
yy=torch.from_numpy(y_train)
xx1=torch.from_numpy(X_test)
yy1=torch.from_numpy(y_test)

model(xx[:10])

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.8089],
        [0.4343],
        [0.3949],
        [0.2892],
        [0.0973],
        [0.4340],
        [0.4008],
        [0.5045],
        [0.2040],
        [0.5758]], grad_fn=<SigmoidBackward0>)

In [15]:
# number of epochs to train the model
n_epochs = 1000

#yy.unsqueeze(1)
#xx.unsqueeze(1)
model.train() # prep model for training
for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    counter = 0
    
    ###################
    # train the model #
    ###################

    ############# new lines #############
    for i in range(len(X_train)//batch_size) :
      X_batch=X_train[i*batch_size:(i+1)*batch_size]
      y_batch=y_train[i*batch_size:(i+1)*batch_size]
    #####################################

    #for i in range(len(y_train)):
      optimizer.zero_grad()

      ##modyfied
      output = model(xx[i*batch_size:(i+1)*batch_size])
      #yy.unsqueeze(1)
      loss = criterion(output, yy[i*batch_size:(i+1)*batch_size])
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      counter += 1
        # for data, target in train_loader:
        # # clear the gradients of all optimized variables
        # optimizer.zero_grad()
        # # forward pass: compute predicted outputs by passing inputs to the model
        # output = model(data)
        # # calculate the loss
        # loss = criterion(output, target)
        # # backward pass: compute gradient of the loss with respect to model parameters
        # loss.backward()
        # # perform a single optimization step (parameter update)
        # optimizer.step()
        # # update running training loss
        # train_loss += loss.item()*data.size(0) 
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/counter

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 	Training Loss: 0.010185
Epoch: 2 	Training Loss: 0.010184
Epoch: 3 	Training Loss: 0.010183
Epoch: 4 	Training Loss: 0.010182
Epoch: 5 	Training Loss: 0.010181
Epoch: 6 	Training Loss: 0.010180
Epoch: 7 	Training Loss: 0.010179
Epoch: 8 	Training Loss: 0.010178
Epoch: 9 	Training Loss: 0.010177
Epoch: 10 	Training Loss: 0.010176
Epoch: 11 	Training Loss: 0.010175
Epoch: 12 	Training Loss: 0.010174
Epoch: 13 	Training Loss: 0.010173
Epoch: 14 	Training Loss: 0.010172
Epoch: 15 	Training Loss: 0.010171
Epoch: 16 	Training Loss: 0.010170
Epoch: 17 	Training Loss: 0.010169
Epoch: 18 	Training Loss: 0.010168
Epoch: 19 	Training Loss: 0.010167
Epoch: 20 	Training Loss: 0.010166
Epoch: 21 	Training Loss: 0.010165
Epoch: 22 	Training Loss: 0.010164
Epoch: 23 	Training Loss: 0.010163
Epoch: 24 	Training Loss: 0.010162
Epoch: 25 	Training Loss: 0.010161
Epoch: 26 	Training Loss: 0.010160
Epoch: 27 	Training Loss: 0.010160
Epoch: 28 	Training Loss: 0.010159
Epoch: 29 	Training Loss: 0.0

In [27]:
from sklearn.metrics import accuracy_score
output=model(xx1)
s=0
for i in range (len(y_test)) :
  s+= (yy1[i]-output[i])**2
print(s/len(y_test))
print(s)

tensor([0.0076], grad_fn=<DivBackward0>)
tensor([0.7704], grad_fn=<AddBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
